# Criptografía sin miedo 🔐

Este notebook acompaña al video y artículo educativo sobre los fundamentos criptográficos de la blockchain.

## Contenidos:
- Hashing con SHA-256
- Firma digital con criptografía asimétrica
- Árbol de Merkle (concepto base)


## 1. Hash con SHA-256

In [1]:
import hashlib

def hash_sha256(text):
    return hashlib.sha256(text.encode()).hexdigest()

mensaje = "Hola blockchain"
hash_resultado = hash_sha256(mensaje)
print("Mensaje:", mensaje)
print("Hash SHA-256:", hash_resultado)

Mensaje: Hola blockchain
Hash SHA-256: 2b9b5e8eea748971d74144b3562c5224a44b0b596694b526250864516012b232


## 2. Firma digital con ECDSA (usando curva SECP256k1)

In [11]:
from ecdsa import SigningKey, SECP256k1

# Crear clave privada y pública
clave_privada = SigningKey.generate(curve=SECP256k1)
clave_publica = clave_privada.verifying_key

# Mostrar la clave privada y pública
print("Clave privada:", clave_privada.to_string().hex())
print("Clave pública:", clave_publica.to_string().hex())

# Firmar un mensaje (codificado a bytes)
mensaje = "Transferencia de 1 BTC a María"
mensaje_bytes = mensaje.encode('utf-8')  # Codifica el string a bytes

firma = clave_privada.sign(mensaje_bytes)

# Verificar firma
es_valida = clave_publica.verify(firma, mensaje_bytes)

print("Mensaje:", mensaje)
print("Firma válida:", es_valida)


Clave privada: cc1cbe9bd838b6dc24652fa91a80cbbb37abf9eeb0694901328c83d35409d852
Clave pública: 964ed450cd0391856913a66daab06f6a6180505521f4e2981e8b83694ac481350184089d4d7e9fa711d78a0b7fc85dd418a3f48e5588319996572f9b840843d3
Mensaje: Transferencia de 1 BTC a María
Firma válida: True


## 3. Árbol de Merkle básico (2 niveles)

#### Es una estructura que permite verificar que una transacción está incluida en un bloque… ¡sin necesidad de leer todo el bloque!

#### Es ideal para mantener seguridad, pero con eficiencia.

            raiz_merkle
           /           \
         h12           h34
        /   \         /   \
      tx1  tx2     tx3   tx4


In [14]:
import hashlib

def hash_sha256(data):
    return hashlib.sha256(data.encode()).hexdigest()

def hash_doble(a, b):
    return hashlib.sha256((a + b).encode()).hexdigest()

# Transacciones base
tx1 = hash_sha256("A")
tx2 = hash_sha256("B")
tx3 = hash_sha256("C")
tx4 = hash_sha256("D")

# Hash de pares
h12 = hash_doble(tx1, tx2)
h34 = hash_doble(tx3, tx4)

# Raíz de Merkle
raiz_merkle = hash_doble(h12, h34)
print("Raíz de Merkle:", raiz_merkle)

# --- Verificación de inclusión de tx1 ---
# Merkle proof: [tx2, h34]  => orden importa: izquierda/derecha

def verificar_merkle(tx_hash, proof, raiz, posiciones):
    actual = tx_hash
    for i in range(len(proof)):
        if posiciones[i] == 'derecha':
            actual = hash_doble(actual, proof[i])
        else:
            actual = hash_doble(proof[i], actual)
    return actual == raiz

# Construimos la prueba para tx1
proof_tx1 = [tx2, h34]
posiciones_tx1 = ['derecha', 'derecha']  # tx1 izquierda de tx2, luego h12 izquierda de h34

# Verificamos
es_valido = verificar_merkle(tx1, proof_tx1, raiz_merkle, posiciones_tx1)
print("¿tx1 está en el árbol?:", es_valido)


Raíz de Merkle: 50a504831bd50fee3581d287168a85a8dcdd6aa777ffd0fe35e37290268a0153
¿tx1 está en el árbol?: True
